## 3+ema+Channel+Strategy+Backtesting+(c)

In [3]:
#pip install pandas_datareader

In [4]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd 
import pandas_datareader.data as pdr
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

init_notebook_mode(connected = True)
cf.go_offline()

In [5]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

In [6]:
def ema_3channel_backtesting(ticker, previous_days):
    
    start = dt.datetime.today()-dt.timedelta(previous_days)
    end = dt.datetime.today()
    global ohlc_intraday
    ohlc_intraday = {}
    ohlc_intraday[ticker] = yf.download(ticker,start,end)
    ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
    global df_signal
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Open','High','Low','Close']]
    ema50=pd.DataFrame(calculate_ema(df_signal["Close"],50),index=np.arange(50,len(df_signal["Close"])+1),columns=['ema50'])
    ema100=pd.DataFrame(calculate_ema(df_signal["Close"],100),index=np.arange(100,len(df_signal["Close"])+1),columns=['ema100'])
    ema150=pd.DataFrame(calculate_ema(df_signal["Close"],150),index=np.arange(150,len(df_signal["Close"])+1),columns=['ema150'])
    df_signal = pd.concat([df_signal,ema50,ema100,ema150],axis=1,join='outer')[:-1]
    df_signal['Low_t-2']=df_signal['Low'].shift(2)
    df_signal['ema50_t-2']=df_signal['ema50'].shift(2)
    df_signal['Close_t-1']=df_signal['Close'].shift(1)
    df_signal['ema50_t-1']=df_signal['ema50'].shift(1)
    df_signal['ema100_t-1']=df_signal['ema100'].shift(1)
    df_signal['ema150_t-1']=df_signal['ema150'].shift(1)

    condition_t_2 = (df_signal['Low_t-2']<df_signal['ema50_t-2'])
    condition_t_1 = (df_signal['Close_t-1']>df_signal['ema50_t-1']) & (df_signal['ema50_t-1']>df_signal['ema100_t-1']) & (df_signal['ema100_t-1']>df_signal['ema150_t-1'])

    buy_condition = (condition_t_2) & (condition_t_1) 
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')
    df_signal = df_signal[['Date','Open','High','Low','Close','ema50','ema100','ema150','Buy']]
    df_signal['Action'] = ''
    
    condition = (df_signal['Buy']=='Buy')
    df_signal2 = df_signal[condition].copy()
    df_signal2 = df_signal2.reset_index(drop = True)
    df_signal2 = df_signal2[['Date','Buy']]
    
    columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
    global df_Trading
    df_Trading = pd.DataFrame(columns = columns_Trading)
    
    Accu_profit = 0
    for i in range(len(df_signal2)):
        check_action = df_signal[df_signal['Date']==df_signal2['Date'][i]].reset_index()
        if check_action['Action'][0] =='':
            date_index = df_signal[df_signal['Date']==df_signal2['Date'][i]].index[0]
            Date_Buy = df_signal['Date'][date_index]
            Price_Buy = df_signal['Open'][date_index]
            stop_loss = 0.92*Price_Buy
            df_signal.loc[date_index,'Action']='Buy'
            date_index += 1
            stage = 'ToStop'
            close_order = False
            while True & (date_index < len(df_signal)):
                df_signal['Action'][date_index] = 'Hold'
                if stage == 'ToStop':
                    if df_signal['Close'][date_index] < stop_loss:
                        close_order = True
                        break
                    elif df_signal['Close'][date_index] > Price_Buy*1.05:
                        stage = 'ToProfit'
                elif stage == 'ToProfit':  
                    if df_signal['Close'][date_index] < df_signal['ema50'][date_index]:
                        close_order = True
                        break
                    elif df_signal['Close'][date_index] < Price_Buy*1.05:
                        stage = 'ToStop'
                date_index += 1
                if date_index >= len(df_signal):
                    break
            if close_order:
                date_index += 1
                df_signal['Action'][date_index] = 'Sell'
                Date_Sell = df_signal['Date'][date_index]
                Price_Sell = df_signal['Open'][date_index]
                fee = 0.002*(Price_Sell+Price_Buy)
                PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
                Holding_Days = abs(Date_Sell-Date_Buy).days
                Accu_profit += PL

                Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],columns = columns_Trading)
                df_Trading = df_Trading.append(Add_Trading)

    df_Trading = df_Trading.reset_index(drop = True)
    df_Trading = df_Trading.round(2)
    total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
    cagr = ((((total_profit+100)/100)**(1/(len(df_signal)/250)))-1)*100
    win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
    loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
    win_rate = win_trade*100/(win_trade+loss_trade)
    avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
    avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
    expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
    list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
    global df_summary
    df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
    
def plot_3ema():

    fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
    specs=[[{"type": "scatter"}],
          [{"type": "table"}],[{"type": "table"}]])

    buy = df_Trading[['Date_Buy','Price_Buy']].copy()
    sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
    fig.add_trace(go.Candlestick(x=df_signal['Date'],
                open=df_signal['Open'],
                high=df_signal['High'],
                low=df_signal['Low'],
                close=df_signal['Close']),row=1,col=1)

    #fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['Close'],mode='lines',name='Close'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema50'],mode='lines',name='ema50'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema100'],mode='lines',name='ema100'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema150'],mode='lines',name='ema150'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)
    
    fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

    fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

    fig.update_layout( height=900, showlegend=True, title_text='3 ema Channel',)
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.show()

In [7]:
ticker = 'AMZN'
ema_3channel_backtesting(ticker, previous_days = 4000)

[*********************100%***********************]  1 of 1 completed


In [8]:
plot_3ema()

In [9]:
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2011-08-30,205.78,2011-10-04,209.62,1.46,35,1.46
1,2011-10-06,220.28,2011-10-27,204.26,-7.66,21,-6.20
2,2012-06-01,208.44,2012-07-26,220.00,5.13,55,-1.06
3,2012-07-27,225.25,2012-10-11,248.00,9.68,76,8.62
4,2012-10-18,247.77,2012-10-26,228.60,-8.12,8,0.50
5,2012-11-23,239.89,2013-02-12,259.19,7.63,81,8.13
6,2013-02-14,267.37,2013-08-16,286.55,6.76,183,14.89
7,2013-09-04,288.33,2013-10-10,304.63,5.24,36,20.13
8,2013-10-11,304.77,2014-01-30,393.77,28.74,111,48.87
9,2014-01-31,371.76,2014-03-28,340.05,-8.91,56,39.96


In [10]:
# 1. Download Date from yfinance
ticker = 'AAPL'
start = dt.datetime.today()-dt.timedelta(4000)
end = dt.datetime.today()
ohlc_intraday = {}
ohlc_intraday[ticker] = yf.download(ticker,start,end)
ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
df_signal = ohlc_intraday[ticker].copy()
df_signal

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,10.144161,309080800
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,10.227140,255519600
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,10.218874,300428800
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,10.292056,311931200
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,10.485884,448560000
...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,179.449997,115228100
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,175.740005,153237000
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,174.330002,139380400
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,179.300003,131063300


In [11]:
# 2.remove some columns
df_signal = df_signal[['Date','Open','High','Low','Close']]
df_signal

,Date,Open,High,Low,Close
0,2011-01-04,11.872857,11.875000,11.719643,11.831786
1,2011-01-05,11.769643,11.940714,11.767857,11.928571
2,2011-01-06,11.954286,11.973214,11.889286,11.918929
3,2011-01-07,11.928214,12.012500,11.853571,12.004286
4,2011-01-10,12.101071,12.258214,12.041786,12.230357
...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003


In [12]:
# 3.Add ema50,ema100,ema150
ema50=pd.DataFrame(calculate_ema(df_signal["Close"],50),index=np.arange(50,len(df_signal["Close"])+1),columns=['ema50'])
ema100=pd.DataFrame(calculate_ema(df_signal["Close"],100),index=np.arange(100,len(df_signal["Close"])+1),columns=['ema100'])
ema150=pd.DataFrame(calculate_ema(df_signal["Close"],150),index=np.arange(150,len(df_signal["Close"])+1),columns=['ema150'])
df_signal = pd.concat([df_signal,ema50,ema100,ema150],axis=1,join='outer')[:-1]
df_signal

,Date,Open,High,Low,Close,ema50,ema100,ema150
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,155.585627,150.051211,145.716493
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,156.521484,150.633365,146.163295
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,157.275152,151.130526,146.555039
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,157.943969,151.589922,146.922919


In [13]:
# 4. Find long 'buy' day

df_signal['Low_t-2']=df_signal['Low'].shift(2)
df_signal['ema50_t-2']=df_signal['ema50'].shift(2)
df_signal['Close_t-1']=df_signal['Close'].shift(1)
df_signal['ema50_t-1']=df_signal['ema50'].shift(1)
df_signal['ema100_t-1']=df_signal['ema100'].shift(1)
df_signal['ema150_t-1']=df_signal['ema150'].shift(1)

condition_t_2 = (df_signal['Low_t-2']<df_signal['ema50_t-2'])
condition_t_1 = (df_signal['Close_t-1']>df_signal['ema50_t-1']) & \
               (df_signal['ema50_t-1']>df_signal['ema100_t-1']) & \
               (df_signal['ema100_t-1']>df_signal['ema150_t-1'])

buy_condition = (condition_t_2) & (condition_t_1) 
df_signal['Buy']=np.where(buy_condition, 'Buy', '')

df_signal

,Date,Open,High,Low,Close,ema50,ema100,ema150,Low_t-2,ema50_t-2,Close_t-1,ema50_t-1,ema100_t-1,ema150_t-1,Buy
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN,NaN,NaN,11.831786,NaN,NaN,NaN,
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN,11.719643,NaN,11.928571,NaN,NaN,NaN,
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN,11.767857,NaN,11.918929,NaN,NaN,NaN,
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN,11.889286,NaN,12.004286,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,155.585627,150.051211,145.716493,170.699997,153.983863,174.559998,154.811162,149.556084,145.329332,
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,156.521484,150.633365,146.163295,173.919998,154.811162,179.449997,155.585627,150.051211,145.716493,
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,157.275152,151.130526,146.555039,174.690002,155.585627,175.740005,156.521484,150.633365,146.163295,
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,157.943969,151.589922,146.922919,175.529999,156.521484,174.330002,157.275152,151.130526,146.555039,


In [14]:
# 5. remove some columns and add 'Action' column
df_signal = df_signal[['Date','Open','High','Low','Close','ema50','ema100','ema150','Buy']]
df_signal['Action'] = ''
df_signal

,Date,Open,High,Low,Close,ema50,ema100,ema150,Buy,Action
0,2011-01-04,11.872857,11.875000,11.719643,11.831786,NaN,NaN,NaN,,
1,2011-01-05,11.769643,11.940714,11.767857,11.928571,NaN,NaN,NaN,,
2,2011-01-06,11.954286,11.973214,11.889286,11.918929,NaN,NaN,NaN,,
3,2011-01-07,11.928214,12.012500,11.853571,12.004286,NaN,NaN,NaN,,
4,2011-01-10,12.101071,12.258214,12.041786,12.230357,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...
2753,2021-12-10,175.210007,179.630005,174.690002,179.449997,155.585627,150.051211,145.716493,,
2754,2021-12-13,181.119995,182.130005,175.529999,175.740005,156.521484,150.633365,146.163295,,
2755,2021-12-14,175.250000,177.740005,172.210007,174.330002,157.275152,151.130526,146.555039,,
2756,2021-12-15,175.110001,179.500000,172.309998,179.300003,157.943969,151.589922,146.922919,,


In [15]:
# 6. filter days 'buy' signal 
condition = (df_signal['Buy']=='Buy')
df_signal2 = df_signal[condition].copy()
df_signal2 = df_signal2.reset_index(drop = True)
df_signal2 = df_signal2[['Date','Buy']]
df_signal2

,Date,Buy
0,2011-08-10,Buy
1,2011-08-11,Buy
2,2011-08-12,Buy
3,2011-08-24,Buy
4,2011-08-25,Buy
...,...,...
137,2021-09-24,Buy
138,2021-09-27,Buy
139,2021-10-18,Buy
140,2021-10-19,Buy


In [16]:
# 7. Make trading dataframe
columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
df_Trading = pd.DataFrame(columns = columns_Trading)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit


In [17]:
# 8.Trading loop and record in df_Trading
Accu_profit = 0
for i in range(len(df_signal2)):
    check_action = df_signal[df_signal['Date']==df_signal2['Date'][i]].reset_index()
    if check_action['Action'][0] =='':
        date_index = df_signal[df_signal['Date']==df_signal2['Date'][i]].index[0]
        Date_Buy = df_signal['Date'][date_index]
        Price_Buy = df_signal['Open'][date_index]
        stop_loss = 0.92*Price_Buy
        df_signal.loc[date_index,'Action']='Buy'
        date_index += 1
        stage = 'ToStop'
        close_order = False
        while True & (date_index < len(df_signal)):
            df_signal['Action'][date_index] = 'Hold'
            if stage == 'ToStop':
                if df_signal['Close'][date_index] < stop_loss:
                    close_order = True
                    break
                elif df_signal['Close'][date_index] > Price_Buy*1.05:
                    stage = 'ToProfit'
            elif stage == 'ToProfit':  
                if df_signal['Close'][date_index] < df_signal['ema50'][date_index]:
                    close_order = True
                    break
                elif df_signal['Close'][date_index] < Price_Buy*1.05:
                    stage = 'ToStop'
            date_index += 1
            if date_index >= len(df_signal):
                break
        if close_order:
            date_index += 1
            df_signal['Action'][date_index] = 'Sell'
            Date_Sell = df_signal['Date'][date_index]
            Price_Sell = df_signal['Open'][date_index]
            fee = 0.002*(Price_Sell+Price_Buy)
            PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
            Holding_Days = abs(Date_Sell-Date_Buy).days
            Accu_profit += PL
            
            Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL, \
                                Holding_Days,Accu_profit]],columns = columns_Trading)
            df_Trading = df_Trading.append(Add_Trading)
            
df_Trading = df_Trading.reset_index(drop = True)
df_Trading = df_Trading.round(2)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2011-08-10,13.26,2011-10-03,13.58,2.08,54,2.08
1,2011-10-11,14.02,2012-04-25,21.99,56.31,197,58.39
2,2012-04-26,21.94,2012-05-15,20.05,-8.98,19,49.41
3,2012-05-24,20.57,2012-10-08,23.10,11.91,137,61.31
4,2014-01-08,19.24,2014-09-26,24.63,27.55,261,88.86
5,2014-09-29,24.66,2014-12-15,27.67,11.79,77,100.65
6,2014-12-19,28.07,2015-05-01,31.52,11.90,133,112.56
7,2015-05-04,32.38,2015-08-04,29.35,-9.71,92,102.85
8,2016-11-22,27.99,2017-06-13,36.79,30.99,203,133.84
9,2017-07-14,36.99,2017-09-21,38.95,4.88,69,138.72


In [18]:
#9. Trading Summary
total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
cagr = ((((total_profit+100)/100)**(1/(len(df_signal)/250)))-1)*100
win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
win_rate = win_trade*100/(win_trade+loss_trade)
avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]

df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
df_summary

,Measurement,Result
0,Total Profit,277.95
1,CAGR,12.81%
2,Win Trade,17
3,Loss Trade,5
4,Win Rate,77.27%
5,Average Gain,19.0%
6,Average loss,-9.0%
7,Expected Return,21.55%


In [19]:
#10.Plot graph

fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
specs=[[{"type": "scatter"}],
        [{"type": "table"}],[{"type": "table"}]])

buy = df_Trading[['Date_Buy','Price_Buy']].copy()
sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
fig.add_trace(go.Candlestick(x=df_signal['Date'],
            open=df_signal['Open'],
            high=df_signal['High'],
            low=df_signal['Low'],
            close=df_signal['Close']),row=1,col=1)

#fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['Close'],mode='lines',name='Close'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema50'],mode='lines',name='ema50'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema100'],mode='lines',name='ema100'),row= 1,col =1)
fig.add_trace(go.Scatter(x = df_signal['Date'], y=df_signal['ema150'],mode='lines',name='ema150'),row= 1,col =1)
fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)

fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

fig.update_layout( height=900, showlegend=True, title_text='3 ema Channel',)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()